In [1]:
%matplotlib widget
import numpy as np
import pylab as plt
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)
plt.style.use('default')
# from antipasti.mpl import ensure_print

In [2]:
from ipywidgets import HBox, IntSlider, FloatSlider
from scipy.integrate import solve_ivp, odeint

## Define phase space drawer

In [3]:
class PhaseSpace:
    def __init__(self, fig, ax):
        # stash the Line2D artist
        self.ln = ax[0].plot([], [], 'kx')[0]
        self.ax = ax
        self.fig = fig
#         self.ax[0].annotate('Left-click to add points', (.5, .9), 
#                       ha='center', xycoords='axes fraction')
        # register our method to be called per-click
        self.button_cid = fig.canvas.mpl_connect('button_press_event',
                                                       self.on_button)

        self.real_lines = []
        self.phase_lines = []
        
        # configure axes
        self.ax[0].set_xlim([-2.05, 2.05])
        self.ax[0].set_ylim([-2.05, 2.05])
        self.ax[0].plot([-2, 2], [0, 0], 'k-')
        self.ax[0].plot([0, 0], [-2, 2], 'k-')
        self.ax[0].set_xlabel('$v_x$')
        self.ax[0].set_ylabel('$v_y$')
        self.ax[1].set_xlabel('$x$')
        self.ax[1].set_ylabel('$y$')
        
        # configure slider
        self.K = 0
        self.m = 1
        self.g = 9.81
        self.tspan = np.linspace(0, 10, 500)
        self.slider_k = FloatSlider(
            orientation='horizontal', description='k: ',
            value=0, min=0, max=10, step = 0.1
        )

        self.slider_k.observe(self.update_k, names='value')
        self.max_y = 0
        
    def update_k(self, change):
        self.K = change.new
        
    def on_button(self, event):
#         print(f'button: {event.button!r} @ ({event.xdata}, {event.ydata}) + key: {event.key}')
         # only consider events from the lines Axes or if not the left mouse button bail! 
        if event.inaxes is not self.ln.axes or event.button != 1:
            return
   
        if event.key == 'shift':
            for line in self.real_lines:
                line.remove()
            for line in self.phase_lines:
                line.remove()
            self.real_lines = []
            self.phase_lines = []
            self.ln.set_data([], [])
            
        else:
            Y0 = [0, 1, event.xdata, event.ydata]
            def derivative(t, y):
                return [y[2], y[3],
                        -self.K/self.m*y[2],
                        -self.K/self.m*y[3] - self.g
                       ]
            def hit_ground(t, y): return y[1] + 1
            hit_ground.terminal = True
            sol = solve_ivp(derivative, [self.tspan[0], self.tspan[-1]], Y0, t_eval=self.tspan,
                           events=hit_ground
                           )

            # append the new point to the current Line2D data
            xdata = list(self.ln.get_xdata()) + [event.xdata]
            ydata = list(self.ln.get_ydata()) + [event.ydata]

            # and update the data on the Line2D artist
            self.ln.set_data(xdata, ydata)
            self.real_lines.append(self.ax[1].plot(sol.y[0], sol.y[1])[0])
            self.phase_lines.append(self.ax[0].plot(sol.y[2], sol.y[3])[0])
            
            # y lims
            self.max_y = max(self.max_y, 1.05 * np.max(self.real_lines[-1].get_ydata()))
            self.ax[1].set_ylim([0, self.max_y])
            
        # ask the UI to re-draw the next time it can
        self.ln.figure.canvas.draw_idle()
        
    @property
    def curve(self):
        # get the current (x, y) for the line
        return {'x': self.ln.get_xdata(), 'y': self.ln.get_ydata()}

In [4]:
fig, ax = figure(1, 1, 2, figsize=(10, 5))

PS = PhaseSpace(fig, ax)
PS.slider_k

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=0.0, description='k: ', max=10.0)